# 🌐 IP Subnetting Practice Tool

This notebook provides an interactive interface to the IP Subnetting Practice Tool, allowing you to perform various subnet calculations and visualize the results.

## Introduction

IP subnetting is the process of dividing an IP network into smaller sub-networks. This is essential for:
- Efficient IP address allocation
- Improved network security
- Reduced network congestion
- Simplified network management

In [ ]:
# Import required modules
import ipaddress
import re
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML

# Add the parent directory to the path to import the SubnetCalculator class
import os
sys.path.append(os.path.abspath(os.path.dirname('')))

# Import the SubnetCalculator class
from subnet_calculator import SubnetCalculator

## 1. Calculate Network Information

Let's start by calculating detailed information about a network. Enter a network in CIDR notation (e.g., 192.168.1.0/24) or with a subnet mask (e.g., 192.168.1.0 255.255.255.0).

In [ ]:
def display_network_info(network_str):
    """Display detailed information about a network in a formatted table."""
    try:
        # Get network information
        info = SubnetCalculator.get_network_info(network_str)
        
        # Create a styled HTML table
        html = """<style>
        .subnet-table {
            width: 100%;
            border-collapse: collapse;
            font-family: Arial, sans-serif;
        }
        .subnet-table th {
            background-color: #4CAF50;
            color: white;
            text-align: left;
            padding: 12px;
        }
        .subnet-table td {
            border: 1px solid #ddd;
            padding: 12px;
        }
        .subnet-table tr:nth-child(even) {
            background-color: #f2f2f2;
        }
        </style>
        <table class='subnet-table'>
            <tr><th colspan='2'>Network Information</th></tr>
        """
        
        # Add rows for each piece of information
        for key, value in info.items():
            html += f"<tr><td>{key}</td><td>{value}</td></tr>"
            
        html += "</table>"
        
        # Display the HTML table
        display(HTML(html))
        
        return info
    
    except ValueError as e:
        print(f"Error: {e}")
        return None

# Example usage
network = "192.168.1.0/24"
info = display_network_info(network)

## 2. Divide a Network into Subnets

Now, let's divide a network into smaller subnets. You can specify either the number of subnets (must be a power of 2) or the new prefix length.

In [ ]:
def display_subnets(network_str, num_subnets=None, new_prefix_length=None):
    """Display a list of subnets in a formatted table."""
    try:
        # Get subnets
        if num_subnets:
            subnets = SubnetCalculator.subnet_network(network_str, num_subnets=num_subnets)
        elif new_prefix_length:
            subnets = SubnetCalculator.subnet_network(network_str, new_prefix_length=new_prefix_length)
        else:
            raise ValueError("Either num_subnets or new_prefix_length must be provided")
        
        # Create a DataFrame for better display
        data = []
        for subnet in subnets:
            host_range = f"{next(subnet.hosts()) if subnet.num_addresses > 1 else subnet.network_address} - " + \
                        f"{list(subnet.hosts())[-1] if subnet.num_addresses > 1 else subnet.broadcast_address}"
            
            data.append({
                'Subnet': str(subnet),
                'Network Address': str(subnet.network_address),
                'Broadcast': str(subnet.broadcast_address),
                'Mask': str(subnet.netmask),
                'Range': host_range,
                'Hosts': subnet.num_addresses - 2 if subnet.prefixlen < 31 else subnet.num_addresses
            })
        
        df = pd.DataFrame(data)
        display(df.style.set_properties(**{'text-align': 'left'}))
        
        # Create a visualization of the subnets
        plt.figure(figsize=(12, 6))
        
        # Create a horizontal bar chart
        y_pos = np.arange(len(subnets))
        plt.barh(y_pos, [subnet.num_addresses for subnet in subnets], align='center', alpha=0.7)
        plt.yticks(y_pos, [str(subnet) for subnet in subnets])
        plt.xlabel('Number of IP Addresses')
        plt.title('Subnet Sizes')
        
        plt.tight_layout()
        plt.show()
        
        return subnets
    
    except ValueError as e:
        print(f"Error: {e}")
        return None

# Example usage - divide a /24 network into 4 subnets
network = "192.168.1.0/24"
num_subnets = 4
subnets = display_subnets(network, num_subnets=num_subnets)

## 3. Find Subnet for Host Count

Let's determine the appropriate subnet mask for a given number of hosts. This is useful when planning a network for a specific number of devices.

In [ ]:
def display_subnet_for_hosts(num_hosts, base_network=None):
    """Display the appropriate subnet for a given number of hosts."""
    try:
        # Find the appropriate prefix length
        prefix_length = SubnetCalculator.find_subnet_for_hosts(num_hosts)
        
        # Calculate the subnet mask
        subnet_mask = str(ipaddress.IPv4Network(f'0.0.0.0/{prefix_length}').netmask)
        
        # Calculate the number of hosts
        max_hosts = 2**(32-prefix_length) - 2 if prefix_length < 31 else 2**(32-prefix_length)
        
        # Display the result
        print(f"For {num_hosts} hosts, you need a /{prefix_length} subnet (netmask: {subnet_mask})")
        print(f"This subnet can accommodate {max_hosts} hosts")
        
        # If a base network is provided, display detailed information about the resulting network
        if base_network:
            network = ipaddress.IPv4Network(f"{base_network}/{prefix_length}", strict=False)
            info = SubnetCalculator.get_network_info(str(network))
            display_network_info(str(network))
        
        # Create a visualization of host capacity
        plt.figure(figsize=(10, 6))
        
        # Create a bar chart comparing required vs. available hosts
        plt.bar(['Required Hosts', 'Available Hosts'], [num_hosts, max_hosts], color=['blue', 'green'], alpha=0.7)
        plt.ylabel('Number of Hosts')
        plt.title(f'Host Capacity for /{prefix_length} Subnet')
        
        # Add value labels on top of bars
        for i, v in enumerate([num_hosts, max_hosts]):
            plt.text(i, v + 0.1, str(v), ha='center')
        
        plt.tight_layout()
        plt.show()
        
        return prefix_length
    
    except ValueError as e:
        print(f"Error: {e}")
        return None

# Example usage - find a subnet for 100 hosts
num_hosts = 100
base_network = "192.168.1.0"
prefix_length = display_subnet_for_hosts(num_hosts, base_network)

## 4. Find Supernet

Let's find the smallest supernet that contains multiple networks. This is useful for route summarization.

In [ ]:
def display_supernet(networks):
    """Display the supernet that contains all provided networks."""
    try:
        # Find the supernet
        supernet = SubnetCalculator.get_supernet(networks)
        
        if supernet:
            print(f"Supernet that contains all provided networks: {supernet}")
            info = display_network_info(str(supernet))
            
            # Create a visual representation of the supernet and its networks
            network_objects = [ipaddress.IPv4Network(net) for net in networks]
            
            plt.figure(figsize=(12, 6))
            
            # Create a horizontal bar chart
            y_pos = np.arange(len(network_objects) + 1)
            sizes = [net.num_addresses for net in network_objects] + [supernet.num_addresses]
            labels = [str(net) for net in network_objects] + [str(supernet) + " (Supernet)"]
            colors = ['blue'] * len(network_objects) + ['red']
            
            plt.barh(y_pos, sizes, align='center', alpha=0.7, color=colors)
            plt.yticks(y_pos, labels)
            plt.xlabel('Number of IP Addresses (log scale)')
            plt.title('Networks and Supernet Sizes')
            plt.xscale('log')
            
            plt.tight_layout()
            plt.show()
            
            return supernet
        else:
            print("No common supernet found for the provided networks.")
            return None
    
    except ValueError as e:
        print(f"Error: {e}")
        return None

# Example usage - find the supernet for multiple networks
networks = ["172.16.16.0/24", "172.16.17.0/24", "172.16.18.0/24", "172.16.19.0/24"]
supernet = display_supernet(networks)

## 5. Interactive Subnet Calculator

Let's create an interactive subnet calculator using widgets. This allows you to experiment with different subnetting scenarios interactively.

In [ ]:
from ipywidgets import widgets
from IPython.display import clear_output

# Create tabs for different operations
tab = widgets.Tab()

# Network Info Tab
network_info_input = widgets.Text(value='192.168.1.0/24', description='Network:')
network_info_button = widgets.Button(description='Calculate')
network_info_output = widgets.Output()

# Subnet Network Tab
subnet_network_input = widgets.Text(value='192.168.1.0/24', description='Network:')
subnet_method = widgets.RadioButtons(options=['Number of Subnets', 'Prefix Length'], description='Method:')
subnet_num = widgets.IntSlider(value=4, min=2, max=128, step=2, description='Subnets:')
subnet_prefix = widgets.IntSlider(value=26, min=1, max=32, step=1, description='Prefix:')
subnet_button = widgets.Button(description='Calculate')
subnet_output = widgets.Output()

# Hosts Tab
hosts_num = widgets.IntText(value=100, description='Hosts:')
hosts_network = widgets.Text(value='192.168.1.0', description='Base Network:')
hosts_button = widgets.Button(description='Calculate')
hosts_output = widgets.Output()

# Supernet Tab
supernet_networks = widgets.Textarea(value='172.16.16.0/24\n172.16.17.0/24\n172.16.18.0/24\n172.16.19.0/24', 
                                   description='Networks:')
supernet_button = widgets.Button(description='Calculate')
supernet_output = widgets.Output()

# Define callback functions
def on_network_info_button_clicked(b):
    with network_info_output:
        clear_output()
        display_network_info(network_info_input.value)

def on_subnet_button_clicked(b):
    with subnet_output:
        clear_output()
        if subnet_method.value == 'Number of Subnets':
            display_subnets(subnet_network_input.value, num_subnets=subnet_num.value)
        else:
            display_subnets(subnet_network_input.value, new_prefix_length=subnet_prefix.value)

def on_hosts_button_clicked(b):
    with hosts_output:
        clear_output()
        display_subnet_for_hosts(hosts_num.value, hosts_network.value)

def on_supernet_button_clicked(b):
    with supernet_output:
        clear_output()
        networks = [net.strip() for net in supernet_networks.value.split('\n') if net.strip()]
        display_supernet(networks)

# Register callbacks
network_info_button.on_click(on_network_info_button_clicked)
subnet_button.on_click(on_subnet_button_clicked)
hosts_button.on_click(on_hosts_button_clicked)
supernet_button.on_click(on_supernet_button_clicked)

# Create tab contents
tab1 = widgets.VBox([network_info_input, network_info_button, network_info_output])
tab2 = widgets.VBox([subnet_network_input, subnet_method, subnet_num, subnet_prefix, subnet_button, subnet_output])
tab3 = widgets.VBox([hosts_num, hosts_network, hosts_button, hosts_output])
tab4 = widgets.VBox([supernet_networks, supernet_button, supernet_output])

# Set tab titles
tab.children = [tab1, tab2, tab3, tab4]
tab.set_title(0, 'Network Info')
tab.set_title(1, 'Subnet Network')
tab.set_title(2, 'Hosts')
tab.set_title(3, 'Supernet')

# Display the tabs
display(tab)

## Summary

This notebook provides an interactive way to:

1. Calculate detailed information about an IP network
2. Divide a network into smaller subnets
3. Find a subnet that can accommodate a given number of hosts
4. Calculate the supernet that encompasses multiple networks

These tools are essential for network planning, administration, and troubleshooting. The visualizations help to better understand the relationships between networks and their sizes.